In [1]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
from dlomix.models import PrositIntensityPredictor
from prosit_t.models import PrositTransformerIntensityPredictor
from prosit_t.losses import masked_spectral_distance, masked_pearson_correlation_distance
from dlomix.constants import ALPHABET_UNMOD

import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
import keras_nlp
print([x for x in dir(dlomix) if not x.startswith("_")])

2023-06-15 11:08:44.825083: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 11:08:44.932289: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-15 11:08:45.565607: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-15 11:08:45.565673: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

['META_DATA', 'constants', 'data', 'eval', 'layers', 'losses', 'models', 'pipelines', 'reports', 'utils']


In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
DEFAULT_CONFIG = {
    "name": "64_heads_2_encoders_pos_emb_size_512",
    "learning_rate": 0.0001,
    "batch_size": 64,
    "embedding_output_dim": 64,
    "intermediate_dim": 512,
    "transformer_num_heads": 16,
    "mh_num_heads": 16,
    "key_dim": 16,
    "seq_length": 30,
    "len_fion": 6,
    "vocab_dict": ALPHABET_UNMOD,
    "dropout_rate": 0.2,
    "latent_dropout_rate": 0.1,
    "layer_norm_epsilon": 1e-5,
    "num_encoders": 2,
}

In [6]:
project_name = "transforming-prosit"

In [7]:
from prosit_t.data import IntensityDataset

TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/Intensity/proteomeTools_train_val.csv'
BATCH_SIZE = 64
int_data = IntensityDataset(data_source=TRAIN_DATAPATH, seq_length=30,
                            collision_energy_col='collision_energy', batch_size=BATCH_SIZE, val_ratio=0.2, test=False)

2023-06-15 11:08:54.529453: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 11:08:54.932850: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43648 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6


In [ ]:
wandb.init(project=project_name, name=DEFAULT_CONFIG["name"])
cb_wandb = WandbCallback()
callbacks = [cb_wandb]
model = PrositTransformerIntensityPredictor(**DEFAULT_CONFIG)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=masked_spectral_distance,
    metrics=[masked_pearson_correlation_distance],
)
model.fit(
    int_data.train_data,
    validation_data=int_data.val_data,
    epochs=30,
    callbacks=callbacks,
)

wandb: Currently logged in as: mamisashvili-lizi. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/30
16 1 -1 -----------true
32 1 -1 -----------true
21 1 -1 -----------true
24 1 -1 -----------true
41 1 -1 -----------true
8 1 -1 -----------true
15 1 -1 -----------true
10 1 -1 -----------true
8 1 -1 -----------true
7 1 -1 -----------true
13 1 -1 -----------true
12 1 -1 -----------true
12 1 -1 -----------true
8 1 -1 -----------true
18 1 -1 -----------true
15 1 -1 -----------true
14 1 -1 -----------true
16 1 -1 -----------true
13 1 -1 -----------true
16 1 -1 -----------true
8 1 -1 -----------true
10 1 -1 -----------true
12 1 -1 -----------true
10 1 -1 -----------true
18 1 -1 -----------true
7 1 -1 -----------true
11 1 -1 -----------true
18 1 -1 -----------true
40 1 -1 -----------true
15 1 -1 -----------true
11 1 -1 -----------true
9 1 -1 -----------true
9 1 -1 -----------true
7 1 -1 -----------true
17 1 -1 -----------true
10 1 -1 -----------true
13 1 -1 -----------true
10 1 -1 -----------true
11 1 -1 -----------true
7 1 -1 -----------true
11 1 -1 -----------true
10 1 -1 -------

2023-06-15 11:09:01.015582: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


92 0.566476703 0 -----------pred
84 0.410057664 0 -----------pred
84 0.586030841 0 -----------pred
79 0.423374116 0 -----------pred
86 0.31475991 0 -----------pred
93 0.538880467 0 -----------pred
81 0.488995403 0 -----------pred
86 0.364981383 0 -----------pred
86 0.331272691 0 -----------pred
86 0.368758678 0 -----------pred
96 0.356241852 0 -----------pred
80 0.711213112 0 -----------pred
93 0.335661381 0 -----------pred
86 0.302728266 0 -----------pred
86 0.37797609 0 -----------pred
89 0.303585351 0 -----------pred
78 0.302249789 0 -----------pred
90 0.243375435 0 -----------pred
84 0.502640784 0 -----------pred
78 0.327147424 0 -----------pred
76 0.471115977 0 -----------pred
86 0.345179915 0 -----------pred
99 0.37372911 0 -----------pred
90 0.359540224 0 -----------pred
90 0.439675301 0 -----------pred
89 0.683642 0 -----------pred
85 0.563210785 0 -----------pred
86 0.333686262 0 -----------pred
94 0.252433419 0 -----------pred
88 0.254692972 0 -----------pred
76 0.366436869 0

...

22 1 -1 -----------true
15 1 -1 -----------true
36 1 -1 -----------true
17 1 -1 -----------true
11 1 -1 -----------true
19 1 -1 -----------true
45 1 -1 -----------true
85 0.479269326 0 -----------pred
91 0.544513345 0 -----------pred
96 0.761280537 0 -----------pred
93 0.538154185 0 -----------pred
90 0.732845306 0 -----------pred
87 0.703056693 0 -----------pred
84 0.591294527 0 -----------pred
98 0.725129128 0 -----------pred
90 0.732547164 0 -----------pred
91 0.514247715 0 -----------pred
95 0.640733719 0 -----------pred
86 0.528823316 0 -----------pred
89 0.530362248 0 -----------pred
87 0.693792701 0 -----------pred
87 0.673031867 0 -----------pred
85 0.712609887 0 -----------pred
94 0.581921697 0 -----------pred
94 0.793709695 0 -----------pred
82 0.728180408 0 -----------pred
88 0.764551222 0 -----------pred
92 0.526602566 0 -----------pred
94 0.702807069 0 -----------pred
91 0.528547049 0 -----------pred
94 0.531081319 0 -----------pred
86 0.535269 0 -----------pred
89 0.71046

89 0.710377395 0 -----------pred
94 0.762707055 0 -----------pred
86 0.594990909 0 -----------pred
87 0.648796797 0 -----------pred
97 0.505824208 0 -----------pred
92 0.686897397 0 -----------pred
90 0.708427787 0 -----------pred
86 0.772351325 0 -----------pred
93 0.563216269 0 -----------pred
90 0.803377569 0 -----------pred
95 0.884567142 0 -----------pred
84 0.527092159 0 -----------pred
93 0.604793549 0 -----------pred
88 0.635755241 0 -----------pred
91 0.632021666 0 -----------pred
92 0.756345391 0 -----------pred
94 0.580942452 0 -----------pred
83 0.755240917 0 -----------pred
87 0.745921195 0 -----------pred
90 0.625360131 0 -----------pred
97 0.657607913 0 -----------pred
92 0.815263271 0 -----------pred
88 0.759713113 0 -----------pred
85 0.606823206 0 -----------pred
93 0.509249151 0 -----------pred
91 0.694165468 0 -----------pred
83 0.828937531 0 -----------pred
88 0.533970177 0 -----------pred
93 0.681455 0 -----------pred
90 0.730379581 0 -----------pred
95 0.56516313

91 0.739040196 0 -----------pred
91 0.798445761 0 -----------pred
95 0.628177643 0 -----------pred
91 0.738185227 0 -----------pred
86 0.847223 0 -----------pred
91 0.70766151 0 -----------pred
87 0.734060585 0 -----------pred
86 0.659654319 0 -----------pred
92 0.618703246 0 -----------pred
90 0.68212229 0 -----------pred
90 0.756626725 0 -----------pred
81 0.683100581 0 -----------pred
93 0.617949307 0 -----------pred
89 0.625599086 0 -----------pred
92 0.62704289 0 -----------pred
81 0.63504523 0 -----------pred
86 0.537994504 0 -----------pred
84 0.725439131 0 -----------pred
81 0.732908785 0 -----------pred
84 0.60657227 0 -----------pred
85 0.55941838 0 -----------pred
90 0.66538167 0 -----------pred
83 0.56527096 0 -----------pred
 26/368 [=>............................] - ETA: 39s - loss: 0.6862 - masked_pearson_correlation_distance: 0.582310 1 -1 -----------true
35 1 -1 -----------true
13 1 -1 -----------true
5 1 -1 -----------true
11 1 -1 -----------true
30 1 -1 -----------tr

 28/368 [=>............................] - ETA: 39s - loss: 0.6823 - masked_pearson_correlation_distance: 0.576616 1 -1 -----------true
14 1 -1 -----------true
11 1 -1 -----------true
12 1 -1 -----------true
16 1 -1 -----------true
11 1 -1 -----------true
17 1 -1 -----------true
20 1 -1 -----------true


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
model.before_regressor_dense

In [10]:
model.after_regressor_dense

array([[ 0.041415  , -0.4677409 , -0.46987936, ...,  0.11506455,
         0.000692  , -0.07531315],
       [-0.06850231, -0.41201293, -0.5069628 , ..., -0.03013245,
         0.08851966, -0.04154683],
       [-0.01274259, -0.46144944, -0.45356986, ...,  0.12718825,
         0.08634335, -0.05684397],
       ...,
       [-0.0511005 , -0.4380414 , -0.00255323, ...,  0.17445141,
         0.19662704, -0.13639511],
       [-0.22380069, -0.39012387,  0.0476205 , ...,  0.07090019,
         0.04131823, -0.13901782],
       [ 0.18150638, -0.4451025 , -0.55009305, ...,  0.20111981,
        -0.18515666,  0.01678644]], dtype=float32)

In [11]:
model.after_relu

array([[0.041415  , 0.        , 0.        , ..., 0.11506455, 0.000692  ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.08851966,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.12718825, 0.08634335,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.17445141, 0.19662704,
        0.        ],
       [0.        , 0.        , 0.0476205 , ..., 0.07090019, 0.04131823,
        0.        ],
       [0.18150638, 0.        , 0.        , ..., 0.20111981, 0.        ,
        0.01678644]], dtype=float32)

In [ ]:
model.predict(int_data.val_data.take(1))

In [ ]:
wandb.init(project=project_name, name='test_on_csv')

model = PrositIntensityPredictor(seq_length=30,embedding_output_dim=16,
        recurrent_layers_sizes=(256, 512))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [ ]:
model.compile(optimizer=optimizer, 
              loss=masked_spectral_distance, metrics=[masked_pearson_correlation_distance])

In [ ]:
history = model.fit(int_data.train_data, validation_data=int_data.val_data,
                    epochs=60, callbacks=[WandbCallback()])

In [ ]:
wandb.finish()